In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
#from google.colab import files
#uploaded = files.upload()

In [2]:

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train_original=train.copy()
test_original=test.copy()

In [3]:
train.head()

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,150728,2074419,37,1,single,rented,no,Analyst,Kozhikode,Kerala,1,11,0
1,148628,3728026,48,17,single,rented,yes,Official,Nanded,Maharashtra,11,12,0
2,25237,7159687,38,5,single,rented,no,Computer_operator,Jalgaon,Maharashtra,3,11,0
3,110209,277326,45,9,single,rented,no,Economist,Medininagar,Jharkhand,9,13,0
4,187462,8822191,26,13,single,rented,no,Aviator,Gorakhpur,Uttar_Pradesh,3,10,1


In [4]:
test.head()

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years
0,144036,96682,52,16,married,rented,no,Petroleum_Engineer,Mehsana,Gujarat,11,12
1,166701,7478707,39,9,single,rented,no,Design_Engineer,Ranchi,Jharkhand,3,10
2,238331,6018693,53,12,single,rented,no,Army_officer,Bhiwandi,Maharashtra,5,11
3,144552,5282884,72,3,single,rented,no,Politician,Munger,Bihar,3,10
4,203146,9312068,23,0,single,rented,no,Industrial_Engineer,Panipat,Haryana,0,11


In [5]:
train=train.drop(columns = ['Id', 'city', 'state', 'profession', 'current_house_years'],axis=1)
test=test.drop(columns = ['Id', 'city', 'state', 'profession', 'current_house_years'],axis=1)

In [6]:
X = train.drop('risk_flag',axis=1)
y = train.risk_flag

In [7]:
X = pd.get_dummies(X)
train=pd.get_dummies(train)
test=pd.get_dummies(test)
X.head()

,income,age,experience,current_job_years,married_married,married_single,house_ownership_norent_noown,house_ownership_owned,house_ownership_rented,car_ownership_no,car_ownership_yes
0,2074419,37,1,1,False,True,False,False,True,True,False
1,3728026,48,17,11,False,True,False,False,True,False,True
2,7159687,38,5,3,False,True,False,False,True,True,False
3,277326,45,9,9,False,True,False,False,True,True,False
4,8822191,26,13,3,False,True,False,False,True,True,False


In [8]:
smt= SMOTE()
c1=list(X.columns.values)
X, y = smt.fit_resample(X, y)
X = pd.DataFrame(X, columns = c1)
X.head()

,income,age,experience,current_job_years,married_married,married_single,house_ownership_norent_noown,house_ownership_owned,house_ownership_rented,car_ownership_no,car_ownership_yes
0,2074419,37,1,1,False,True,False,False,True,True,False
1,3728026,48,17,11,False,True,False,False,True,False,True
2,7159687,38,5,3,False,True,False,False,True,True,False
3,277326,45,9,9,False,True,False,False,True,True,False
4,8822191,26,13,3,False,True,False,False,True,True,False


In [9]:
paramgrid = {'max_depth': list(range(2,32,2)), 'n_estimators': list(range(10,100,10))}
grid_search=GridSearchCV(RandomForestClassifier(random_state=1),paramgrid)
x_train, x_cv, y_train, y_cv = train_test_split(X,y, test_size=0.15, random_state=1)
grid_search.fit(x_train,y_train)
GridSearchCV(estimator=RandomForestClassifier(random_state=None), param_grid={'max_depth': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32], 'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]})
grid_search.best_estimator_
RandomForestClassifier(n_estimators = 100, max_depth=32, random_state=None, max_features=None)
i=1
mean = 0
kf = StratifiedKFold(n_splits=100, random_state=1, shuffle=True)
for train_index,test_index in kf.split(X,y):
    print ('\n{} of kfold {} '.format(i,kf.n_splits))
    xtr,xvl = X.loc[train_index],X.loc[test_index]
    ytr,yvl = y[train_index],y[test_index]
    model = RandomForestClassifier(n_estimators = 100, random_state=None, max_depth=32, max_features=None)
    model.fit(xtr,ytr)
    pred_test = model.predict(xvl)
    score = accuracy_score(yvl,pred_test)
    mean += score
    print ('accuracy_score',score)
    i+=1
    pred_test = model.predict(test)
    pred = model.predict_proba(xvl)[:,1]
print ('\n Mean Validation Accuracy',mean/(i-1))


1 of kfold 100 
accuracy_score 0.9168564920273349

2 of kfold 100 
accuracy_score 0.9350797266514806

3 of kfold 100 
accuracy_score 0.929384965831435

4 of kfold 100 
accuracy_score 0.9328018223234624

5 of kfold 100 
accuracy_score 0.9271070615034168

6 of kfold 100 
accuracy_score 0.9305239179954442

7 of kfold 100 
accuracy_score 0.9305239179954442

8 of kfold 100 
accuracy_score 0.9191343963553531

9 of kfold 100 
accuracy_score 0.9236902050113895

10 of kfold 100 
accuracy_score 0.9202733485193622

11 of kfold 100 
accuracy_score 0.9066059225512528

12 of kfold 100 
accuracy_score 0.9350797266514806

13 of kfold 100 
accuracy_score 0.9384965831435079

14 of kfold 100 
accuracy_score 0.9259681093394078

15 of kfold 100 
accuracy_score 0.9225512528473804

16 of kfold 100 
accuracy_score 0.9225512528473804

17 of kfold 100 
accuracy_score 0.9304446978335233

18 of kfold 100 
accuracy_score 0.9201824401368301

19 of kfold 100 
accuracy_score 0.928164196123147

20 of kfold 100 
accur

In [10]:
sub = {'risk_flag':1, 'Id':2}
sub['risk_flag']=pred_test
sub['Id']=test_original['Id']
pd.DataFrame(sub, columns=['Id', 'risk_flag']).to_csv('grid.csv')